` !pip install webvtt-py ` https://github.com/glut23/webvtt-py

`!pip install XlsxWriter`

Delete empty text after timestamp in the sbv files.

In [7]:
import pandas as pd
import numpy as np
import webvtt
from datetime import datetime
from difflib import SequenceMatcher

#Chinese sbv file
InOriginal = "C:\\Users\\Jiachen\\OneDrive\\YouTube Subtitles\\Youtube LeTV Published\\4509 Templates with Chinese subs\\Empress In The Palace (YouTube Template) - E21 - converted.sbv"
#English translation
InTranslation = "C:\\Users\\Jiachen\\OneDrive\\YouTube Subtitles\\Youtube LeTV Published\\EP21LucyOCT10.sbv"
#Revised translation
InRevised = "C:\\Users\\Jiachen\\OneDrive\\YouTube Subtitles\\Youtube LeTV Published\\EP21CarsenOCT10.sbv"
#Output file
OutFile = "C:\\Users\\Jiachen\\OneDrive\\YouTube Subtitles\\测试 培训\\Carsen修改前后对比\\EP21LucyCarsen20191010.xlsx"

In [8]:
def sbv2df(sbv,textCol):
    """ 
    Store (start, end, and text) of each time segment in the sbv file in a row of a pandas dataframe.
    Input args 
        sbv (string): the file path of an sbv file
        textCol (string): the name of the text column
    """
    data = []
    global webvtt
    webvtt = webvtt.from_sbv(sbv)
    for caption in webvtt:
        data.append({'start':datetime.strptime(caption.start,'%H:%M:%S.%f').time(), 
                     'end':datetime.strptime(caption.end,'%H:%M:%S.%f').time(),
                     textCol:caption.text})
    df = pd.DataFrame(data)
    df = df.replace('\n',' ', regex=True)
    df = df[['start','end',textCol]] 
    return df   

In [9]:
original = sbv2df(InOriginal,"Chinese")
translation = sbv2df(InTranslation,"Translation")
revised = sbv2df(InRevised,"Revised")

In [10]:
o = original.set_index(['start','end'])
t = translation.set_index(['start','end'])
r = revised.set_index(['start','end'])
output = t.join(r, how='outer')
output[13:23]

,,Translation,Revised
start,end,,
00:02:17,00:02:22,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...
00:02:23.240000,00:02:27.300000,"Lady Sourire, what brings you here in this dee...","Lady Sourire, what brings you here on this fro..."
00:02:27.740000,00:02:31.660000,It's just that the heavy frost makes it hard f...,The heavy frost makes it hard for Lingrong to ...
00:02:32.020000,00:02:34.560000,NaN,so I came to accompany her.
00:02:32.160000,00:02:34.160000,so I came to accompany her.,NaN
00:02:34.940000,00:02:36.820000,NaN,Sourire is an excellent zither player.
00:02:35.780000,00:02:40.400000,Lady Sourire plays Guqin very well. I invited ...,NaN
00:02:36.820000,00:02:41.020000,NaN,I invited her for fear that my singing would b...
00:02:45.880000,00:02:47.880000,Singing accompanied by the Guqin is naturally...,NaN


```python
lastStart = output.index[0][0]
for index, row in output.iterrows():
    if index[0] != output.index[0][0]:  #not datetime.time(0, 0)
        if index[0] != lastStart: #different start time from last time segment
            lastStart = index[0]
            print("different start time")
            print(index[0],index[1])
            print(row[0],row[1],'\n')
        elif index[0] == lastStart: #same start time with last time segment
            print("same start time")
            print(index[0],index[1])
            print(row[0],row[1],'\n')
```
Example output below

different start time
00:02:25.180000 00:02:28.620000
I bow in respect before Lady Sourire. nan 

different start time
00:02:26.420000 00:02:29.860000
nan I bow in respect before Lady Sourire. 

different start time
00:02:30.820000 00:02:32.580000
What are you carrying, Mr Jiang? nan 

same start time
00:02:30.820000 00:02:33.480000
nan What is it, sir, that you have to carry personally? 

In [11]:
# For each time sgegment with the same start time, replace Translation or Revised with the last non-NaN value

lastStart = output.index[0][0]
for index, row in output.iterrows():
    if index[0] != output.index[0][0]:  #not datetime.time(0, 0)
        if index[0] != lastStart: #different start time from last time segment
            lastStart = index[0]
            lastTranslation = row[0]
            lastRevised = row[1]
        elif index[0] == lastStart: #same start time with last time segment
            if pd.isna(row[0]) and ~pd.isna(lastTranslation):
                row[0] = lastTranslation
            if pd.isna(row[1]) and ~pd.isna(lastRevised):
                row[1] = lastRevised
output[13:23]

,,Translation,Revised
start,end,,
00:02:17,00:02:22,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...
00:02:23.240000,00:02:27.300000,"Lady Sourire, what brings you here in this dee...","Lady Sourire, what brings you here on this fro..."
00:02:27.740000,00:02:31.660000,It's just that the heavy frost makes it hard f...,The heavy frost makes it hard for Lingrong to ...
00:02:32.020000,00:02:34.560000,NaN,so I came to accompany her.
00:02:32.160000,00:02:34.160000,so I came to accompany her.,NaN
00:02:34.940000,00:02:36.820000,NaN,Sourire is an excellent zither player.
00:02:35.780000,00:02:40.400000,Lady Sourire plays Guqin very well. I invited ...,NaN
00:02:36.820000,00:02:41.020000,NaN,I invited her for fear that my singing would b...
00:02:45.880000,00:02:47.880000,Singing accompanied by the Guqin is naturally...,NaN


In [12]:
# Clean output (df): drop the rows like the following in the above output
    # 00:02:30.820000	00:02:32.580000
    # 00:02:34.100000	00:02:36.720000
#But keep the rows like 
    # 00:02:25.180000	00:02:28.620000
    # 00:02:26.420000	00:02:29.860000

lastStart = output.index[0][0]
clean=output
for index, row in output.iterrows():
    if index[0] != output.index[0][0]:  #not datetime.time(0, 0)
        if index[0] != lastStart: #different start time from last time segment
            lastStart = index[0]      
            #print("START",index[0],'\n',output.loc[index[0]].shape)  #to find out the pattern in shape
            if output.loc[index[0]].shape[0] == 2:   #one start (index) matches 2 end (index)
                clean.drop((index[0],index[1]), inplace=True)
clean[13:23]

,,Translation,Revised
start,end,,
00:02:17,00:02:22,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...
00:02:23.240000,00:02:27.300000,"Lady Sourire, what brings you here in this dee...","Lady Sourire, what brings you here on this fro..."
00:02:27.740000,00:02:31.660000,It's just that the heavy frost makes it hard f...,The heavy frost makes it hard for Lingrong to ...
00:02:32.020000,00:02:34.560000,NaN,so I came to accompany her.
00:02:32.160000,00:02:34.160000,so I came to accompany her.,NaN
00:02:34.940000,00:02:36.820000,NaN,Sourire is an excellent zither player.
00:02:35.780000,00:02:40.400000,Lady Sourire plays Guqin very well. I invited ...,NaN
00:02:36.820000,00:02:41.020000,NaN,I invited her for fear that my singing would b...
00:02:45.880000,00:02:48.760000,Singing accompanied by the Guqin is naturally...,Singing accompanied by the zither is naturally...


In [13]:
#Calculate word change ratio (0-1) in clean (df)
    #word change ratio (0-1) with 1 indicating most different Revision from Translation 
    #However, 1 is usually due to unmatching timestamps
    #A ratio value over 0.6 (1-ratio<0.4) means the sequences are close matches
clean['WordChange'] =  np.nan
for idx,row in clean.iterrows():
    if pd.isna(row['Translation']):
        row['Translation'] = ''
    if pd.isna(row['Revised']):
        row['Revised'] = ''
    clean.loc[idx,'WordChange'] = 1- SequenceMatcher(None,row['Translation'],row['Revised']) .ratio()
clean[13:23]

,,Translation,Revised,WordChange
start,end,,,
00:02:17,00:02:22,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...,0.000000
00:02:23.240000,00:02:27.300000,"Lady Sourire, what brings you here in this dee...","Lady Sourire, what brings you here on this fro...",0.059829
00:02:27.740000,00:02:31.660000,It's just that the heavy frost makes it hard f...,The heavy frost makes it hard for Lingrong to ...,0.131783
00:02:32.020000,00:02:34.560000,NaN,so I came to accompany her.,1.000000
00:02:32.160000,00:02:34.160000,so I came to accompany her.,NaN,1.000000
00:02:34.940000,00:02:36.820000,NaN,Sourire is an excellent zither player.,1.000000
00:02:35.780000,00:02:40.400000,Lady Sourire plays Guqin very well. I invited ...,NaN,1.000000
00:02:36.820000,00:02:41.020000,NaN,I invited her for fear that my singing would b...,1.000000
00:02:45.880000,00:02:48.760000,Singing accompanied by the Guqin is naturally...,Singing accompanied by the zither is naturally...,0.089286


In [14]:
output = o.join(clean, how='outer')
output[13:23]

,,Chinese,Translation,Revised,WordChange
start,end,,,,
00:02:17,00:02:22,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...,[contact translators] carsenwei@yahoo.com [sub...,0.000000
00:02:22.640000,00:02:23.470000,莞贵人,NaN,NaN,NaN
00:02:23.240000,00:02:27.300000,NaN,"Lady Sourire, what brings you here in this dee...","Lady Sourire, what brings you here on this fro...",0.059829
00:02:24.440000,00:02:26.510000,这夜深霜重的你怎么也来了,NaN,NaN,NaN
00:02:27.080000,00:02:28.390000,正因夜深霜浓,NaN,NaN,NaN
00:02:27.740000,00:02:31.660000,NaN,It's just that the heavy frost makes it hard f...,The heavy frost makes it hard for Lingrong to ...,0.131783
00:02:29.040000,00:02:30.550000,陵容妹妹独步难行,NaN,NaN,NaN
00:02:30.840000,00:02:33.190000,所以臣妾特来与妹妹做伴,NaN,NaN,NaN
00:02:32.020000,00:02:34.560000,NaN,NaN,so I came to accompany her.,1.000000


In [15]:
#Reset index so that "start" and "end" will appear in the Excel file
df = output.reset_index(level=['start','end'])

# Write to Excel file with formats
writer = pd.ExcelWriter(OutFile, engine='xlsxwriter') #https://xlsxwriter.readthedocs.io/index.html
df.to_excel(writer, sheet_name='Sheet1', index=False)
# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Set the column width and format.
format1 = workbook.add_format({'text_wrap': True})
worksheet.set_column('A:B', 12)
worksheet.set_column('C:E', 38, format1)
worksheet.set_column('F1:F1048576', 5)

# Conditional formatting based on word change %
    # https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html
    #colors https://xlsxwriter.readthedocs.io/working_with_colors.html

# Green fill with dark green text.
format2 = workbook.add_format({'bg_color':   '#C6EFCE',
                               'font_color': '#006100'})
# Light red fill with dark red text.
format3 = workbook.add_format({'bg_color':   '#FFC7CE',
                               'font_color': '#9C0006'})

# Light yellow fill with dark yellow text.
format4 = workbook.add_format({'bg_color':   '#FFEB9C',
                               'font_color': '#9C6500'})


worksheet.conditional_format('F1:F1048576', {'type':     'cell',
                                        'criteria': '<',
                                        'value':    0.4,
                                        'format':   format2})

worksheet.conditional_format('F1:F1048576', {'type':     'cell',
                                        'criteria': 'between',
                                        'minimum':  0.4,
                                        'maximum':  0.99,
                                        'format':   format3})

worksheet.conditional_format('F1:F1048576', {'type':     'cell',
                                        'criteria': '>',
                                        'value':    0.99,
                                        'format':   format4})

# Close the Pandas Excel writer and output the Excel file.
writer.save()



Other functions that might be useful to do the cleaning

```python
output.count(level='start')[12:19] #a df of 1s and 0s
output.count(level='start').iloc[14] #This is a pandas series
output.count(level='start').iloc[14].name #datetime.time(0, 2, 25, 180000)
0 in output.count(level='start').iloc[14].values #True

idx022518 = output.count(level='start').iloc[14].name
output.loc[idx022518] #df with 1 row

idx023082 = output.count(level='start').iloc[16].name
output.loc[idx023082] #df with 2 rows

df = output.loc[idx023082]
df.fillna(method='ffill') #forward fill; 'bfill' for backward fill
```

Iterate a MultiIndex DataFrame by index and row

```python
for index, row in output.iterrows():
    print(index[0])
    print(type(row[0]),type(row[1]))
    print(row[0])
    print(row[1],"\n")
```
